## Create Raster from Array

A tutorial that shows how to create a test image using pixel values from an array. This type of images are useful to test various algorithms and prepare tutorials showing the effect of them.

We use xarray to create a DataArray. Since the DataArray must be georeferenced, we use the `rioxarray` extension to assign a CRS and create a GeoTIFF image.

## Setup and Data Download

The following blocks of code will install the required packages and download the datasets to your Colab environment.

In [ ]:
try:
    import rioxarray
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        !pip install rioxarray --quiet
    else:
        print('rioxarray not found, please install via conda in your environment')

In [ ]:
import os
import numpy as np
import xarray as xr
import rioxarray as rxr

In [ ]:
output_folder = 'output'

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

We want to create an image with a resolution of 1000 meters with the upper-left pixel at coordinates (780850,1432187)

In [ ]:
upper_left_x, upper_left_y = (780850,1432187)
resolution = 1000
num_pixels = 4

We want to create a 4x4 image. Define a 2-dimentional array. Since we are storing small integers set the data type to **Byte** (uint8). 

In [ ]:
array = np.array([
    [0, 0, 1, 1],
    [0, 0, 1, 1],
    [0, 2, 2, 2],
    [2, 2, 3, 3]
], dtype=np.uint8)

If you wanted a larger array with random values, you can try something like below.
```
num_pixels = 10000
array = np.random.randint(0, 4, size=(num_pixels, num_pixels)).astype(np.uint8)
```

Define an array of coordinates. We need X and Y coordinates for each pixels.
We use `np.linspace` function to create a sequence of x and y coordinates for each pixel of the image.

In [ ]:
x_coords = np.linspace(
    start=upper_left_x,
    stop=upper_left_x + (resolution*(num_pixels-1)),
    num=num_pixels, dtype=np.uint)
y_coords = np.linspace(
    start=upper_left_y,
    stop=upper_left_y - (resolution*(num_pixels-1)),
    num=num_pixels, dtype=np.uint)
x_coords, y_coords

(array([780850, 781850, 782850, 783850], dtype=uint64),
 array([1432187, 1431187, 1430187, 1429187], dtype=uint64))

In [ ]:
da = xr.DataArray(
    data=array,
    coords={
        'y': y_coords,
        'x': x_coords
    }
)
da

<xarray.DataArray (y: 4, x: 4)>
array([[0, 0, 1, 1],
       [0, 0, 1, 1],
       [0, 2, 2, 2],
       [2, 2, 3, 3]], dtype=uint8)
Coordinates:
  * y        (y) uint64 1432187 1431187 1430187 1429187
  * x        (x) uint64 780850 781850 782850 783850

The image coordinates we used were from the CRS UTM Zone 43N - EPSG:32643. The `rioxarray` extension provides a `rio` accessor that allows us to set a CRS. Even though we are not using rioxarray directly, we still need to import it which activates the `rio` accessor in xarray.

In [ ]:
da.rio.write_crs('EPSG:32643', inplace=True)
da

<xarray.DataArray (y: 4, x: 4)>
array([[0, 0, 1, 1],
       [0, 0, 1, 1],
       [0, 2, 2, 2],
       [2, 2, 3, 3]], dtype=uint8)
Coordinates:
  * y            (y) uint64 1432187 1431187 1430187 1429187
  * x            (x) uint64 780850 781850 782850 783850
    spatial_ref  int64 0

Now we can save the DataArray in any of the supported raster formats.

In [ ]:
output_file = 'image.tif'
output_path = os.path.join(output_folder, output_file)

In [ ]:
da.rio.to_raster(output_path)